In [ ]:
import os

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch
from tqdm.auto import tqdm
import xarray as xr

from fair import FAIR
from fair.io import read_properties
from fair.interface import initialise, fill

In [ ]:
configs = [
    "ACCESS-ESM1-5",
    "BCC-CSM2-MR",
    "CESM2",
    "CNRM-ESM2-1",
    "CanESM5",
    "GFDL-ESM4",
    "IPSL-CM6A-LR",
    "MIROC-ES2L",
    "MPI-ESM1-2-LR",
    "NorESM2-LM",
    "UKESM1-0-LL",
]

In [ ]:
# NB: rU and rA are in GtC units, we need to convert to GtCO2
data = np.array(
    [
        [
            36.73854601035055,
            25.589821019851797,
            40.704695982343765,
            38.09182601398885,
            35.70573492682388,
            34.26732262345922,
            32.223599635483424,
            33.39478016647172,
            33.33937342916488,
            40.735872526405046,
            37.91594456570033,
        ],
        [
            0.0349535801301073,
            0.00597614250950862,
            0.010664893971021883,
            0.0005810081769186404,
            -0.005958784801017192,
            0.021861410870304354,
            0.016608701817126814,
            0.013104461258272693,
            0.031043773610946346,
            0.009471296196005063,
            0.020138272127751655,
        ],
        [
            3.036651884848311,
            5.196160258410032,
            1.2786398011433562,
            2.472206604249436,
            -0.10385375927186047,
            4.855081881723322,
            1.0693983052255476,
            3.4644393974775767,
            1.499323874009292,
            1.5631932779473914,
            2.6714005898495543,
        ],
        [
            -0.0006603263192310749,
            0.004393751681079472,
            0.004211308668836011,
            0.009783189726962682,
            0.018116906645659014,
            -0.004242277713558451,
            0.012336113500092338,
            0.003993779169272571,
            -0.002570300844565665,
            0.004887468785878646,
            0.0018119017134572424,
        ],
    ]
)
data[1, :] = data[1, :] / 44.009 * 12.011
data[3, :] = data[3, :] / 44.009 * 12.011


cc_params = pd.DataFrame(data.T, columns=["r0", "rU", "rT", "rA"], index=configs)
cc_params

In [ ]:
df = pd.read_csv("../data/4xCO2_cummins_ebm3_cmip6.csv")

multi_runs = {
    "GISS-E2-1-G": "r1i1p1f1",
    "GISS-E2-1-H": "r1i1p3f1",
    "MRI-ESM2-0": "r1i1p1f1",
    "EC-Earth3": "r3i1p1f1",
    "FIO-ESM-2-0": "r1i1p1f1",
    "CanESM5": "r1i1p2f1",
    "FGOALS-f3-L": "r1i1p1f1",
    "CNRM-ESM2-1": "r1i1p1f2",
}

data = np.ones((11, 11)) * np.nan

for im, model in enumerate(configs):
    if model in multi_runs:
        condition = (df["model"] == model) & (df["run"] == multi_runs[model])
    else:
        condition = df["model"] == model
        
        
    data[im, 0] = df.loc[condition, "gamma"].values[0]
    data[im, 1:4] = df.loc[
        condition, "C1":"C3"
    ].values.squeeze()
    data[im, 4:7] = df.loc[condition, "kappa1":"kappa3"].values.squeeze()
    data[im, 7] = df.loc[condition, "epsilon"].values[0]
    data[im, 8] = df.loc[condition, "sigma_eta"].values[0]
    data[im, 9] = df.loc[condition, "sigma_xi"].values[0]
    data[im, 10] = df.loc[condition, "F_4xCO2"].values[0]

eb_params = pd.DataFrame(data, columns=["gamma", "c1", "c2", "c3", "kappa1", "kappa2", "kappa3", "epsilon", "sigma_eta", "sigma_xi", "F_4xCO2"], index=configs)
eb_params

In [ ]:
pd.options.display.max_columns = 50
df_configs = pd.read_csv('../data/calibrated_constrained_parameters_v1.1.0.csv', index_col=0)
valid_all = df_configs.index
df_configs

In [ ]:
df_solar = pd.read_csv(
    "../data/solar_erf_timebounds.csv", index_col="year"
)
df_volcanic = pd.read_csv(
    "../data/volcanic_ERF_monthly_-950001-201912.csv"
)

In [ ]:
volcanic_forcing = np.zeros(551)
for i, year in enumerate(np.arange(1750, 2021)):
    volcanic_forcing[i] = np.mean(
        df_volcanic.loc[
            ((year - 1) <= df_volcanic["year"]) & (df_volcanic["year"] < year)
        ].erf
    )
volcanic_forcing[271:281] = np.linspace(1, 0, 10) * volcanic_forcing[270]

In [ ]:
solar_forcing=np.zeros(551)
solar_forcing[:281] = df_solar['erf'].values[:281]
solar_forcing[271:281] = np.linspace(1, 0, 10) * solar_forcing[271:281]

In [ ]:
trend_shape = np.ones(551)
trend_shape[:271] = np.linspace(0, 1, 271)

In [ ]:
df_methane = pd.read_csv(
    "../data/CH4_lifetime.csv",
    index_col=0,
)

In [ ]:
f = FAIR(ch4_method="Thornhill2021")
# species, properties = read_properties()
# f.define_species(species, properties)
f.define_configs(configs)
# f.allocate()

In [ ]:
species, properties = read_properties()
species.remove('CO2 FFI')      # c-driven run
#species.remove('CO2 AFOLU')    # c-driven run
species.remove('Halon-1202')   # not in CMIP6 list of species
#species.remove('Contrails')    # not modelled in UKESM, I think
#species.remove('NOx aviation') # which renders this redundant
#species.remove('Light absorbing particles on snow and ice')  # I believe not modelled in UKESM
del properties['CO2 FFI']
#del properties['CO2 AFOLU']
del properties['Halon-1202']
#del properties['Contrails']
#del properties['NOx aviation']
#del properties['Light absorbing particles on snow and ice']
for specie in species:
    if properties[specie]['greenhouse_gas']:
        properties[specie]['input_mode'] = 'concentration'

In [ ]:
f.define_time(1750, 2300, 1)
scenarios = ['ssp119', 'ssp534-over', 'ssp585']
f.define_scenarios(scenarios)
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
f.fill_from_rcmip()

In [ ]:
# solar and volcanic forcing
fill(
    f.forcing,
    volcanic_forcing[:, None, None] * df_configs.loc[:, "scale Volcanic"].mean(),
    specie="Volcanic",
)
fill(
    f.forcing,
    solar_forcing[:, None, None] * df_configs.loc[:, "solar_amplitude"].mean()
    + trend_shape[:, None, None] * df_configs.loc[:, "solar_trend"].mean(),
    specie="Solar",
)

# climate response
fill(f.climate_configs["ocean_heat_capacity"], eb_params.loc[:, "c1":"c3"].values)
fill(
    f.climate_configs["ocean_heat_transfer"],
    eb_params.loc[:, "kappa1":"kappa3"].values,
)
fill(f.climate_configs["deep_ocean_efficacy"], eb_params.loc[:,"epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], eb_params.loc[:,"gamma"].values.squeeze())
fill(f.climate_configs["sigma_eta"], eb_params.loc[:,"sigma_eta"].values.squeeze())
fill(f.climate_configs["sigma_xi"], eb_params.loc[:,"sigma_xi"].values.squeeze())
fill(f.climate_configs["seed"], 0)
fill(f.climate_configs["stochastic_run"], False)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], eb_params.loc[:,"F_4xCO2"])

# species level
f.fill_species_configs()

# carbon cycle
fill(f.species_configs["iirf_0"], cc_params.loc[:,"r0"].values.squeeze(), specie="CO2")
fill(
    f.species_configs["iirf_airborne"], cc_params.loc[:,"rA"].values.squeeze(), specie="CO2"
)
fill(f.species_configs["iirf_uptake"], cc_params.loc[:,"rU"].values.squeeze(), specie="CO2")
fill(
    f.species_configs["iirf_temperature"],
    cc_params.loc[:,"rT"].values.squeeze(),
    specie="CO2",
)

# aerosol indirect
fill(f.species_configs["aci_scale"], np.median(df_configs.loc[:,"beta"]))
fill(
    f.species_configs["aci_shape"],
    np.median(df_configs.loc[:,"shape Sulfur"]),
    specie="Sulfur",
)
fill(
    f.species_configs["aci_shape"], np.median(df_configs.loc[:,"shape BC"]), specie="BC"
)
fill(
    f.species_configs["aci_shape"], np.median(df_configs.loc[:,"shape OC"]), specie="OC"
)

# methane lifetime baseline and sensitivity
fill(
    f.species_configs["unperturbed_lifetime"],
    df_methane.loc["historical_best", "base"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "CH4"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "N2O"],
    specie="N2O",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "VOC"],
    specie="VOC",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "NOx"],
    specie="NOx",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "HC"],
    specie="Equivalent effective stratospheric chlorine",
)
fill(
    f.species_configs["lifetime_temperature_sensitivity"],
    df_methane.loc["historical_best", "temp"],
)

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people
# might wanna run pulse expts with these gases)
fill(f.species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
fill(f.species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
fill(f.species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")

# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
    "Equivalent effective stratospheric chlorine",
]:
    fill(
        f.species_configs["erfari_radiative_efficiency"],
        df_configs.loc[:,f"ari {specie}"].mean(),
        specie=specie,
    )

# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Contrails",
    "Light absorbing particles on snow and ice",
    "Land use",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[valid_all,f"scale {specie}"].mean(),
        specie=specie,
    )

for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[valid_all,"scale minorGHG"].mean(),
        specie=specie,
    )

# ozone
for specie in [
    "CH4",
    "N2O",
    "Equivalent effective stratospheric chlorine",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f.species_configs["ozone_radiative_efficiency"],
        df_configs.loc[:,f"o3 {specie}"].mean(),
        specie=specie,
    )

# tune down volcanic efficacy
fill(f.species_configs["forcing_efficacy"], 0.6, specie="Volcanic")


# # initial condition of CO2 concentration (but not baseline for forcing calculations)
# fill(
#     f.species_configs["baseline_concentration"],
#     df_configs.loc[valid_all,"co2_concentration_1750"].values.squeeze(),
#     specie="CO2",
# )

#initialise(f.concentration, f.concentration[])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

f.run()

In [ ]:
pl.plot(f.temperature[:, 0, :, 0]);

In [ ]:
pl.plot(f.temperature[:, 1, :, 0]);

In [ ]:
pl.plot(f.temperature[:, 2, :, 0]);
pl.ylim(-2, 17)

In [ ]:
ds = xr.Dataset(
    data_vars = dict(
        temperature = (["timebound", "scenario", "config"], f.temperature[:, :, :, 0].data),
        forcing = (["timebound", "scenario", "config"], f.forcing_sum.data),
    ),
    coords = dict(
        timebound=np.arange(1750, 2301),
        scenario=scenarios,
        config=configs
    )
)

In [ ]:
ds

In [ ]:
os.makedirs('../results', exist_ok=True)

In [ ]:
ds.to_netcdf('../results/conc-cmip6.nc')

In [ ]:
pl.plot(f.forcing_sum[:, 2, :]);

In [ ]:
pl.plot(f.concentration[:, 2, :, 1]);

In [ ]:
pl.plot(f.forcing[:, 2, :, 1]);